In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


data_path = r"C:\Users\armaa\Downloads\Dataset"


# Parameters
img_height = 224
img_width = 224
batch_size = 32

# Load training and validation sets (80/20 split)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode = "rgb",
    labels = "inferred"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode = "rgb",
    labels = "inferred"
)

print("Total training batches:", tf.data.experimental.cardinality(train_ds))

import os
from PIL import Image

def check_images(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            path = os.path.join(root, file)
            try:
                img = Image.open(path)
                img.verify()
            except:
                print("Removing corrupted file:", path)
                os.remove(path)

check_images(data_path)

class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image,label):
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
    return image,label

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)


num_classes = 6

def one_hot_labels(image, label):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_ds = train_ds.map(one_hot_labels, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(one_hot_labels, num_parallel_calls=AUTOTUNE)


train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)


from collections import Counter

extensions = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        ext = file.split('.')[-1].lower()
        extensions.append(ext)

print(Counter(extensions))


print(f"Classes identified: {class_names}")

Found 812 files belonging to 6 classes.
Using 650 files for training.
Found 812 files belonging to 6 classes.
Using 162 files for validation.
Total training batches: tf.Tensor(21, shape=(), dtype=int64)
Counter({'jpg': 812})
Classes identified: ['HDPE', 'LDPE', 'PET', 'PP', 'PS', 'PVC']


In [10]:
data_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomTranslation(0.1,0.1),
])

base_model = tf.keras.applications.EfficientNetV2B2(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3),
    pooling = 'avg',
)

base_model.trainable = False

model = tf.keras.Sequential([
    data_aug,
    base_model,
    layers.BatchNormalization(),
    layers.Dense(384,activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetv2-b2 (Functional)       │ (None, 1408)                │       8,769,374 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,769,374 (33.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,769,374 (33.45 MB)

In [11]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics = ['accuracy']
)
history = model.fit(train_ds,validation_data=val_ds,epochs=20,callbacks = [callback])

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 588ms/step - accuracy: 0.2815 - loss: 2.3199 - val_accuracy: 0.5370 - val_loss: 1.4810
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 379ms/step - accuracy: 0.4831 - loss: 1.5205 - val_accuracy: 0.5988 - val_loss: 1.2885
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 435ms/step - accuracy: 0.5323 - loss: 1.4363 - val_accuracy: 0.6358 - val_loss: 1.1884
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 416ms/step - accuracy: 0.6185 - loss: 1.1940 - val_accuracy: 0.7037 - val_loss: 1.1118
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 447ms/step - accuracy: 0.6800 - loss: 1.0540 - val_accuracy: 0.7099 - val_loss: 1.0538
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 431ms/step - accuracy: 0.6923 - loss: 0.9977 - val_accuracy: 0.7160 - val_loss: 1.0136
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 446ms/step - accuracy: 0.7262 - loss: 0.9248 - val_accuracy: 0.7222 - val_loss: 0.9760
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 419ms/step - accuracy: 0.7308 - loss: 0.9206 - val_accuracy: 0

In [12]:
base_model.trainable = True

for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(5e-6),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

model.fit(train_ds, validation_data=val_ds, epochs=15,callbacks = [callback])


Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 23s 617ms/step - accuracy: 0.8046 - loss: 0.7027 - val_accuracy: 0.7531 - val_loss: 0.8773
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 429ms/step - accuracy: 0.8385 - loss: 0.6780 - val_accuracy: 0.7407 - val_loss: 0.8831
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 435ms/step - accuracy: 0.8385 - loss: 0.6689 - val_accuracy: 0.7469 - val_loss: 0.8798
Epoch 4/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 436ms/step - accuracy: 0.8492 - loss: 0.6552 - val_accuracy: 0.7346 - val_loss: 0.8821
Epoch 5/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 431ms/step - accuracy: 0.8308 - loss: 0.6777 - val_accuracy: 0.7407 - val_loss: 0.8818
Epoch 6/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 436ms/step - accuracy: 0.8585 - loss: 0.6416 - val_accuracy: 0.7469 - val_loss: 0.8839


In [16]:
model.save("plastic_model_3.keras")